In [4]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone

## 1. Get the airport data from the cities of interest 

In [3]:
#Fuction to get all the airport data from the cities

def get_airports(latitudes, longitudes):
    
    # API headers
    headers = {
      "X-RapidAPI-Key": ,
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    querystring = {"withFlightInfoOnly": "true"}
    
    # DataFrame to store results
    all_airports = []
    
    for lat, lon in zip(latitudes, longitudes):
        # Construct the URL with the latitude and longitude
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"

        # Make the API request
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json()
            airports = pd.json_normalize(data.get('items', []))
            all_airports.append(airports)
        
    return pd.concat(all_airports, ignore_index=True)

In [4]:
# Example coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

get_airports(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
2,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
3,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
4,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
5,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
6,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
7,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
8,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
9,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333


In [5]:
#Final function to get the iata airports codes

def iata_airports_codes(cities_info):
    
    #Create lists
    iata_codes = []
    city_ids = []
    airport_name = []
    
    for index, row in cities_info.iterrows():
        latitude = row['Latitude']
        longitude = row['Longitude']
        
        #Set and call de API
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        querystring = {"lat":latitude,
                       "lon":longitude,
                       "radiusKm":"50",
                       "limit":"10",
                       "withFlightInfoOnly":"true"}
        
        headers = {
            "X-RapidAPI-Key": '687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b',
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.request("GET", url, headers=headers, params=querystring)
        
        if response.status_code == 200:
            airport_json = response.json()
        else:
            return response.reason
        
        #Get the data for each city
        for item in airport_json['items']:
            
            iata_codes.append(item['iata'])
            airport_name.append(item['name'])
            city_ids.append(row['City_id'])
            
    airports_df = pd.DataFrame({
        'iata_code': iata_codes,
        'airport_name': airport_name,
        'City_id':city_ids})
    
    return airports_df

In [6]:
#Read cities_data from SQL - functions

# Create a function to create the connection string
def create_connection_string(): 
    schema = "cities_gans" #Name of the database in MySQL
    host = "127.0.0.1"     #Local host IP
    user = "root"          #Database username
    password = "Focasql9"
    port = 3306
    
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Read the 'cities_data' table into a DataFrame
def read_cities_data(connection_string):
    return pd.read_sql("cities_data", con=connection_string)

In [14]:
#Create the connection string
connection_string = create_connection_string()

#Read the 'citis_data' table from MySQL into a DataFrame
cities_info = read_cities_data(connection_string)
cities_info

,City_id,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.40500
1,2,Hamburg,Germany,53.5500,10.00000
2,3,Munich,Germany,48.1375,11.57500
3,4,Cologne,Germany,50.9364,6.95278
4,5,Paris,France,48.8567,2.35222
5,6,London,United Kingdom,51.5072,-0.12750
6,7,Madrid,Spain,40.4169,-3.70333
7,8,Copenhagen,Denmark,55.6761,12.56830
8,9,Vienna,Austria,48.2083,16.37250
9,10,Budapest,Hungary,47.4925,19.05140


In [15]:
airports_df = iata_airports_codes(cities_info)
airports_df

,iata_code,airport_name,City_id
0,TXL,Berlin -Tegel,1
1,BER,Berlin Brandenburg,1
2,HAM,Hamburg,2
3,MUC,Munich,3
4,CGN,Cologne Bonn,4
5,DUS,Düsseldorf,4
6,LBG,Paris -Le Bourget,5
7,ORY,Paris -Orly,5
8,CDG,Paris Charles de Gaulle,5
9,LCY,London City,6


In [9]:
#Function to send the data to SQL
def save_airports_data(airports_df,connection_string):
    airports_df.to_sql(name = 'cities_airports',
                      if_exists = 'append', #Append new rows
                      con = connection_string,
                      index = False)

In [107]:
#Send airports information to SQL
save_airports_data(airports_df,connection_string)

## 2. Get the arrivals information for the airports

In [94]:
#Set the hours to get the flights data

iata = 'BER'

#Beginning hour: start of tomorros
hour0 = (datetime.today()+timedelta(hours=24))
#First 12 hours: beginning hour + 12 hours
hour12 = hour0 + timedelta(hours=12)
#Last 12 hours of tomorrow: First 12 hours + 12 hours
hour24 = hour12 + timedelta(hours=12)

#Set the start time for the api request in the expected format
start_time = hour0.strftime("%Y-%m-%dT%H:%M")
#Set the end time for the api request in the expected format
end_time = hour12.strftime("%Y-%m-%dT%H:%M")

#Construct the url with the first 12 hours of tomorrow
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"
querystring = {"withLeg":"true",
               "direction":"arrival",
               "withCancelled":"true",
               "withCodeshared":"true",
               "withCargo":"false",
               "withPrivate":"false",
               "withLocation":"false"}

headers = {
    "X-RapidAPI-Key": '687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b',
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

# Make the API request
response = requests.get(url, headers=headers, params=querystring)

flights_json = response.json()
flights_json['arrivals'][0]

{'departure': {'airport': {'icao': 'EDDF',
   'iata': 'FRA',
   'name': 'Frankfurt-am-Main',
   'timeZone': 'Europe/Berlin'},
  'scheduledTime': {'utc': '2025-05-24 10:45Z',
   'local': '2025-05-24 12:45+02:00'},
  'revisedTime': {'utc': '2025-05-24 10:45Z',
   'local': '2025-05-24 12:45+02:00'},
  'terminal': '1',
  'checkInDesk': '259-335',
  'gate': 'A',
  'quality': ['Basic', 'Live']},
 'arrival': {'scheduledTime': {'utc': '2025-05-24 11:55Z',
   'local': '2025-05-24 13:55+02:00'},
  'revisedTime': {'utc': '2025-05-24 11:55Z',
   'local': '2025-05-24 13:55+02:00'},
  'terminal': '1',
  'gate': 'B20',
  'baggageBelt': 'B3',
  'quality': ['Basic', 'Live']},
 'number': 'UA 8887',
 'status': 'Expected',
 'codeshareStatus': 'IsCodeshared',
 'isCargo': False,
 'aircraft': {'reg': 'D-AILK', 'modeS': '3C658B', 'model': 'Airbus A319'},
 'airline': {'name': 'United', 'iata': 'UA', 'icao': 'UAL'}}

In [30]:
flights_json["arrivals"][0].keys()

dict_keys(['departure', 'arrival', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [45]:
#Set the structure of the Data Frame -0 is the item

#arrival airport
flights_json['arrivals'][0]['departure']['airport']['iata']#.get('iata') #iata departure airport


'CTA'

In [77]:
berlin_timezone = timezone('Europe/Berlin')

all_flights = []
now_berlin = datetime.now(berlin_timezone)
iata = 'CDG' 
hour0 = (now_berlin + timedelta(days=1))
hour12 = hour0 + timedelta(hours=12)
# Last 12 hours of tomorrow: First 12 hours + 12 hours
hour24 = hour12 + timedelta(hours=12)

#Set the start time for the api request in the expected format
start_time = hour0.strftime("%Y-%m-%dT%H:%M")
#Set the end time for the api request in the expected format
end_time = hour12.strftime("%Y-%m-%dT%H:%M")

#Construct the url with the first 12 hours of tomorrow
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"

querystring = {"withLeg":"true",
               "direction":"arrival",
               "withCancelled":"true",
               "withCodeshared":"true",
               "withCargo":"false",
               "withPrivate":"false",
               "withLocation":"false"}
headers = {
    "X-RapidAPI-Key": '687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b',
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

# Make the API request
response_dep_arr = requests.get(url, headers=headers, params=querystring)

if response_dep_arr.status_code == 200:
    flights1_json = response_dep_arr.json()
    
    #Add retrieval time
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
    for item in flights1_json.get('arrivals',[]):
        flight_item = {
                    'arrival_airport_iata':iata,
                    'departure_airport_iata': item['departure']['airport'].get('iata',None),
                    'scheduled_arrival_time': item['arrival']['scheduledTime'].get('local',None),
                    'flight_number': item.get('number',None),
                    'timestamp_flight': retrieval_time
                }
        #Append the arriving flights to the all flights list
        all_flights.append(flight_item)
        
#Update time for the next 12 hours of tomorrow in the expected format
start_time = hour12.strftime("%Y-%m-%dT%H:%M")

end_time = hour24.strftime("%Y-%m-%dT%H:%M")


#Update url with the new times
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"

#Get arrivals for the second fragment of time

# Make the API second request
response_dep_arr2 = requests.get(url, headers=headers, params=querystring)

if response_dep_arr2.status_code == 200:
    flights2_json = response_dep_arr2.json()
    
    for item in flights2_json.get('arrivals',[]):
        
        flight_item2 = {
                    'arrival_airport_iata':iata,
                    'departure_airport_iata': item['departure']['airport'].get('iata',None),
                    'scheduled_arrival_time': item['arrival']['scheduledTime'].get('local',None),
                    'flight_number': item.get('number',None),
                    'timestamp_flight': retrieval_time
                }
        
        all_flights.append(flight_item2) 

pd.DataFrame(all_flights) #292 Ber - CDG 614

,arrival_airport_iata,departure_airport_iata,scheduled_arrival_time,flight_number,timestamp_flight
0,CDG,KEF,2025-05-24 13:00+02:00,FI 542,2025-05-23 12:59:44
1,CDG,JFK,2025-05-24 13:00+02:00,AF 9,2025-05-23 12:59:44
2,CDG,JFK,2025-05-24 13:05+02:00,N0 302,2025-05-23 12:59:44
3,CDG,LAX,2025-05-24 13:05+02:00,DL 290,2025-05-23 12:59:44
4,CDG,BOD,2025-05-24 13:05+02:00,AF 9427,2025-05-23 12:59:44
...,...,...,...,...,...
609,CDG,IST,2025-05-25 12:50+02:00,TK 1823,2025-05-23 12:59:44
610,CDG,MRS,2025-05-25 12:55+02:00,AF 9405,2025-05-23 12:59:44
611,CDG,MPL,2025-05-25 12:55+02:00,AF 7463,2025-05-23 12:59:44
612,CDG,YYZ,2025-05-25 12:55+02:00,AF 387,2025-05-23 12:59:44


In [23]:
#Function to get the arriving flights information
def arriving_flights(airports_df):
    
    #Dataframe to store all the flights of all the airports
    all_flights = []
    
    berlin_timezone = timezone('Europe/Berlin')
    
    iata_airports = airports_df['iata_code'].to_list()
    
    #Loop on each airport in the list parameter
    for iata in iata_airports:

        #Set the hours to get the flights data
        #Beginning hour: start of tomorros
        hour0 = (datetime.now(berlin_timezone)+timedelta(hours=24))
        #First 12 hours: beginning hour + 12 hours
        hour12 = hour0 + timedelta(hours=12)
        #Last 12 hours of tomorrow: First 12 hours + 12 hours
        hour24 = hour12 + timedelta(hours=12)

        #Set the start time for the api request in the expected format
        start_time = hour0.strftime("%Y-%m-%dT%H:%M")
        #Set the end time for the api request in the expected format
        end_time = hour12.strftime("%Y-%m-%dT%H:%M")

        #Construct the url with the first 12 hours of tomorrow
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"

        querystring = {"withLeg":"true",
                    "direction":"arrival",
                    "withCancelled":"true",
                    "withCodeshared":"true",
                    "withCargo":"false",
                    "withPrivate":"false",
                    "withLocation":"false"}

        headers = {
            "X-RapidAPI-Key": '687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b',
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        # Make the API request
        response_dep_arr = requests.get(url, headers=headers, params=querystring)

        if response_dep_arr.status_code == 200:
            flights1_json = response_dep_arr.json()
            
            #Add retrieval time
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
            
            for item in flights1_json.get('arrivals',[]):
                flight_item = {
                    'arrival_airport_iata':iata,
                    'departure_airport_iata': item['departure']['airport'].get('iata',None),
                    'scheduled_arrival_time': item['arrival']['scheduledTime'].get('local',None),
                    'flight_number': item.get('number',None),
                    'timestamp_flight': retrieval_time
                }
                
                #Append the arriving flights to the all flights list
                all_flights.append(flight_item) 

        #Update time for the next 12 hours of tomorrow in the expected format
        start_time = hour12.strftime("%Y-%m-%dT%H:%M")
        end_time = hour24.strftime("%Y-%m-%dT%H:%M")

        #Update url with the new times
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{start_time}/{end_time}"

        #Get arrivals for the second fragment of time

        # Make the API second request
        response_dep_arr2 = requests.get(url, headers=headers, params=querystring)

        if response_dep_arr2.status_code == 200:
            
            flights2_json = response_dep_arr2.json()
            
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
            
            for item in flights2_json.get('arrivals',[]):
                
                flight_item2 = {
                    'arrival_airport_iata':iata,
                    'departure_airport_iata': item['departure']['airport'].get('iata',None),
                    'scheduled_arrival_time': item['arrival']['scheduledTime'].get('local',None),
                    'flight_number': item.get('number',None),
                    'timestamp_flight': retrieval_time
                }
            
                all_flights.append(flight_item2) 
    
    #Convert all_flights list to a DataFrame
    all_flights_df = pd.DataFrame(all_flights)
    
     # Ensure proper datetime format
    all_flights_df['scheduled_arrival_time'] = all_flights_df['scheduled_arrival_time'].str[:-6]
    all_flights_df['scheduled_arrival_time'] = pd.to_datetime(all_flights_df['scheduled_arrival_time'])
    all_flights_df['timestamp_flight'] = pd.to_datetime(all_flights_df['timestamp_flight'])
    
    #Drop duplicates
    all_flights_df = all_flights_df.drop_duplicates()
    
    return all_flights_df

In [24]:
all_flights_df = arriving_flights(airports_df)
all_flights_df

,arrival_airport_iata,departure_airport_iata,scheduled_arrival_time,flight_number,timestamp_flight
0,BER,CDG,2025-05-27 17:15:00,AF 1834,2025-05-26 17:08:09
1,BER,SKG,2025-05-27 17:15:00,A3 512,2025-05-26 17:08:09
2,BER,FCO,2025-05-27 17:20:00,FR 41,2025-05-26 17:08:09
3,BER,HEL,2025-05-27 17:20:00,AY 1437,2025-05-26 17:08:09
4,BER,HEL,2025-05-27 17:20:00,AA 8969,2025-05-26 17:08:09
...,...,...,...,...,...
6365,PRG,MAN,2025-05-28 16:35:00,U2 2229,2025-05-26 17:08:19
6366,PRG,MAD,2025-05-28 16:45:00,QS 1057,2025-05-26 17:08:19
6367,PRG,ICN,2025-05-28 16:45:00,KE 969,2025-05-26 17:08:19
6368,PRG,CDG,2025-05-28 16:45:00,QS 1035,2025-05-26 17:08:19


In [26]:
#Function to send the data to SQL
def save_flights_data(all_flights_df,connection_string):
    
    if not all_flights_df.empty:
        
        all_flights_df.to_sql(name = 'flights',
                      if_exists = 'append', #Append new rows
                      con = connection_string,
                      index = False)
        print(f'Saved {len(all_flights_df)} records to table.')
    else:
        print('No flights data to save')

In [114]:
#Send data to SQL flights table
save_flights_data(all_flights_df_cl,connection_string)

In [10]:
#All in one function
def receive_send_airports_data():
    #1. Create SQL connection string
    connection_string = create_connection_string()
    #2. Read and get cities information from cities_data in SQL
    cities_info = read_cities_data(connection_string)
    #3. Get the iata codes for the airports of the cities in the cities_info
    airports_df = iata_airports_codes(cities_info)
    #4. Save airports data in SQL
    save_airports_data(airports_df,connection_string)
    
    return "Data has been updated"

In [11]:
receive_send_airports_data()

'Data has been updated'

In [28]:
def receive_send_flights_data():
    #1. Get the flights information for the airports in aiports_df
    all_flights_df = arriving_flights(airports_df)
    #2. Save flights data in SQL
    save_flights_data(all_flights_df,connection_string)
    
    return "Data has been updated"

In [29]:
receive_send_flights_data()

Saved 6241 records to table.


'Data has been updated'